In [ ]:
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.ensemble import IsolationForest
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Seleccionar ruta de la data
dataframe = pd.read_csv('../data/modeling/01_raw.csv',encoding='latin_1',infer_datetime_format=True)
print('Nro filas: ' + str(dataframe.shape[0]))


In [ ]:
#Opcional: Filtrar data
dataset= dataframe[(dataframe.MTO_TOTAL_CASH>10000) & (dataframe.PERIODO<=202002) & (dataframe.FLGNACIONALIDAD.notna())]
test= dataframe[(dataframe.MTO_TOTAL_CASH>10000) & (dataframe.PERIODO==202003) & (dataframe.FLGNACIONALIDAD.notna())]

In [ ]:
ds=dataset[['ANTIGUEDADCLI','EDAD','CODPAISNACIONALIDAD','FLGNACIONALIDAD','CATPROFESION','MTO_TOTAL_CASH','MTO_TOTAL','CTD_TOTAL_CASH',
            'MTO_CHQGEN','PORC_CHQGEN','FLGAGENTE','MTO_TOTAL_CASH_ZAED', 'CTD_CASH_ZAED', 'CTD_DIAS_DEBAJOLAVA']]

In [ ]:
ds_test=test[['ANTIGUEDADCLI','EDAD','CODPAISNACIONALIDAD','FLGNACIONALIDAD','CATPROFESION','MTO_TOTAL_CASH','MTO_TOTAL','CTD_TOTAL_CASH',
              'MTO_CHQGEN','PORC_CHQGEN','FLGAGENTE','MTO_TOTAL_CASH_ZAED','CTD_CASH_ZAED','CTD_DIAS_DEBAJOLAVA']]

In [ ]:
ds['EDAD'] = dataset['EDAD'].fillna(dataset['EDAD'].mean())
ds_test['EDAD'] = test['EDAD'].fillna(test['EDAD'].mean())

In [ ]:
ds_exp2=ds[['ANTIGUEDADCLI','EDAD','MTO_TOTAL_CASH','MTO_TOTAL','CTD_TOTAL_CASH','MTO_CHQGEN','PORC_CHQGEN','MTO_TOTAL_CASH_ZAED','CTD_CASH_ZAED','CTD_DIAS_DEBAJOLAVA']]

In [ ]:
#Escogemos contamination = 0.02
ki = 0.02
clf_exp2 = IsolationForest(n_estimators = 1000, contamination = ki, random_state=18, max_samples = 'auto', max_features = 1.0, bootstrap = False)

#Genera modelo
clf_exp2.fit(ds_exp2)
#Corre sobre data
y_pred_exp2 = clf_exp2.predict(ds_exp2)

#Copio dataset
dataset_exp2 = dataset.copy()
#Creo nueva variable 
dataset_exp2['if_pred']=clf_exp2.predict(ds_exp2)
print("cantidad de contaminacion: ",ki)
print(dataset_exp2.groupby(['if_pred']).size())

In [ ]:
#Guardar el modelo
pickle.dump(clf_exp2, open('../src/02_models/isolation_forest_model_39.model', 'wb'))

In [ ]:
dataset_exp2.to_csv(r'../data/modeling/04_predicted.csv', sep='|', index = False)